In [ ]:
!pip install gower

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import ast
import gower

import warnings
warnings.filterwarnings('ignore')


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data-science-dating-project/matchmaking_bert_embeddings_v1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58822 entries, 0 to 58821
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  58822 non-null  int64  
 1   status               58822 non-null  object 
 2   sex                  58822 non-null  object 
 3   orientation          58822 non-null  object 
 4   diet                 58822 non-null  object 
 5   drinks               58822 non-null  object 
 6   drugs                58822 non-null  object 
 7   education            58822 non-null  object 
 8   ethnicity            58822 non-null  object 
 9   height               58822 non-null  float64
 10  job                  58822 non-null  object 
 11  location             58822 non-null  object 
 12  offspring            58822 non-null  object 
 13  pets                 58822 non-null  object 
 14  religion             58822 non-null  object 
 15  smokes               58822 non-null 

In [ ]:
df['bert_embeddings'] = df['bert_embeddings'].apply(ast.literal_eval)

In [ ]:
df.sample()

,age,status,sex,orientation,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,smokes,speaks,speaks_original,combined_essay_cols,bert_embeddings
42772,26,single,m,straight,unknown,socially,unknown,college/university_graduated,white,70.0,other,"belvedere tiburon, california",unknown,unknown,atheism,no,multilingual,"english, spanish","i've graduated from college, i've traveled to ...","[0.030042428523302078, 0.02002497762441635, 0...."


In [ ]:
# print(type(df['bert_embeddings'][0]))

In [ ]:
# # # .reshape(1, -1)
# user_A = df.iloc[407][-1]
# A_embedding = np.array(user_A)

# similarity_results = []                    # list to hold results
# # for x in range(len(df)):               # looping through users
# for i, x in enumerate(range(len(df))):
#   if x != 407:
#     user_B = df.iloc[x][-1]
#     B_embedding = np.array(user_B)         # get the second user's embedding

#     cos_similarity = cosine_similarity(A_embedding.reshape(1, -1), B_embedding.reshape(1, -1))  # compute cosine similarity
#     similarity = float(cos_similarity[0][0])

#     similarity_results.append((x, similarity))

# top_5 = sorted(similarity_results, key=lambda x: x[1], reverse=True)[:5]
# print(top_5)

### Matching based on Essays

In [ ]:
def get_essay_matches(user_index, df, top_n):
  similarity_results = []

  user = df.iloc[user_index][-1]
  user_embedding = np.array(user)

  for i, x in enumerate(range(len(df))):
    if x != user_index:
      user_B = df.iloc[x][-1]
      match_embedding = np.array(user_B)         # get the second user's embedding

      cos_similarity = cosine_similarity(user_embedding.reshape(1, -1), match_embedding.reshape(1, -1))  # compute cosine similarity
      similarity = float(cos_similarity[0][0])

      similarity_results.append((df.index[x], similarity))
  top_matches = sorted(similarity_results, key=lambda x: x[1], reverse=True)[:top_n]
  return top_matches

In [ ]:
print(type(df['bert_embeddings'][0]))

<class 'list'>


In [ ]:
# # testing function
# matches = get_essay_matches(407, df, 5)
# print(matches)

# # making sure output is exactly 5
# assert len(matches)  == 5

# # the user index passed through should not be the one in returned matches
# assert 407 not in [i for i, score in matches]

# # scores are floats
# assert all([isinstance(score, float) for i, score in matches])

# # checking similarity in essays
# for user_id, score in matches:
#     print(f"User {user_id} (Score: {score}):")
#     print(df.iloc[user_id]['combined_essay_cols'][:200])
#     print("------")

### Matching based on traits (Gower's Distance)

In [ ]:
# fixing a mapping issue in the offspring column
df['offspring'] = df['offspring'].replace({'no Kids, might want': 'no kids, might want'})

In [ ]:
# converting object to categorical.
#df[['status', 'sex', 'orientation', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'job', 'location', 'offspring', 'pets', 'religion', 'smokes', 'speaks']] = df[['status', 'sex', 'orientation', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'job', 'location', 'offspring', 'pets', 'religion', 'smokes', 'speaks']].astype('category')

In [ ]:
df_gowers = df.drop(columns=['speaks_original', 'combined_essay_cols', 'bert_embeddings'])
df_gowers.head()

,age,status,sex,orientation,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,smokes,speaks
0,22,single,m,straight,anything,socially,never,college/university_studying,mixed,75.0,transportation,"south san francisco, california","no kids, might want",likes pets,agnosticism,occasionally,monolingual
1,35,single,m,straight,other,often,sometimes,other,white,70.0,hospitality / travel,"oakland, california","no kids, might want",likes pets,agnosticism,no,multilingual
2,38,single,m,straight,anything,socially,unknown,masters_graduated,unknown,68.0,rather not say,"san francisco, california",unknown,has pets,other,no,multilingual
3,23,single,m,straight,vegetarian,socially,unknown,college/university_studying,white,71.0,student,"berkeley, california","no kids, doesn't want",likes cats,other,no,multilingual
4,29,single,m,straight,unknown,socially,never,college/university_graduated,mixed,66.0,artistic / musical / writer,"san francisco, california",unknown,likes pets,other,no,monolingual


In [ ]:
df_gowers.info()
df_gowers.head()
df_gowers.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58822 entries, 0 to 58821
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          58822 non-null  int64  
 1   status       58822 non-null  object 
 2   sex          58822 non-null  object 
 3   orientation  58822 non-null  object 
 4   diet         58822 non-null  object 
 5   drinks       58822 non-null  object 
 6   drugs        58822 non-null  object 
 7   education    58822 non-null  object 
 8   ethnicity    58822 non-null  object 
 9   height       58822 non-null  float64
 10  job          58822 non-null  object 
 11  location     58822 non-null  object 
 12  offspring    58822 non-null  object 
 13  pets         58822 non-null  object 
 14  religion     58822 non-null  object 
 15  smokes       58822 non-null  object 
 16  speaks       58822 non-null  object 
dtypes: float64(1), int64(1), object(15)
memory usage: 7.6+ MB


,0
age,0
status,0
sex,0
orientation,0
diet,0
drinks,0
drugs,0
education,0
ethnicity,0
height,0


In [ ]:
# # testing out gower's distance
# user_A = df_gowers.iloc[[407]]
# print(user_A)

# # #excluding user 407 from the dataframe to show potential matches only
# df_gowers = df_gowers.drop(index=407)
# df_gowers = df_gowers.reset_index(drop=True)

# user_A = user_A[df_gowers.columns]
# # # looking at matrix
# A_distance = gower.gower_matrix(user_A, df_gowers)
# A_similarity = (1 - A_distance)
# print(A_similarity)

In [ ]:
def get_lifestyle_matches(user_index, df, top_n):
  # creating a list to return the matches
  lifestyle_matches = []

  # getting users lifestyle choices
  user = df.iloc[[user_index]]

  # dropping to prevent user from matching with self
  df = df.drop(index=user_index)
  df["original_index"] = df.index
  df = df.reset_index(drop=True)

  # re-aligning columns
  user = user[df.drop(columns="original_index").columns]

  user_distance = gower.gower_matrix(user, df.drop(columns="original_index"))
  similarity = 1 - user_distance

  for i in range(len(df)):
    score = float(similarity[0][i])
    user_id = df["original_index"][i]
    lifestyle_matches.append((user_id, score))

  top_matches = sorted(lifestyle_matches, key=lambda x: x[1], reverse=True)[:top_n]

  return top_matches

In [ ]:
# testing function
# matches = get_lifestyle_matches(user_index=407, df=df_gowers, n=10)
# print(matches)

# making sure output is exactly 5
# assert len(matches)  == 5

# the user index passed through should not be the one in returned matches
# assert 407 not in [i for i, score in matches]

# scores are floats
# assert all([isinstance(score, float) for i, score in matches])

In [ ]:
user_index = 407
user_essay_match = get_essay_matches(user_index, df, 3)
user_lifestyle_match = get_lifestyle_matches(user_index, df_gowers, 3)

print("User Essay Match:\n", user_essay_match)
print("User Lifestyle Match:\n", user_lifestyle_match)

User Essay Match:
 [(37420, 0.5965516440805656), (1815, 0.5826354859246876), (25414, 0.5663942161756832)]
User Lifestyle Match:
 [(np.int64(18992), 0.7923875451087952), (np.int64(50224), 0.7849727869033813), (np.int64(31531), 0.7839841842651367)]


In [ ]:
# why did these essays match?
print("User_A:\n")
print(df.loc[407, 'combined_essay_cols'])

print("Matched user:\n")
print(df.loc[37420, 'combined_essay_cols'])

User_A:

hopefulromantic was taken and 2hopefulromantic was too long... lost the "c". oh well ... where do i start. okay, i am a simple human being, yet with a deep soul, who has a heart of gold and who believes in fairy tales. however, i also believe that the pillars of any successful relationship are understanding, loyalty, honesty, compromise, the will to make things work, communication and respect. i believe in fate too and that people meet for a purpose of higher nature and that there is a reason for everything.  i have waited and continue to wait for my "dream girl" and don't mind waiting my whole life for her. i have promised myself to stay single and not to marry for the sake of being married only; i will marry when i fall madly in love with that special woman who will make my heart miss a beat every time i see her or even just think of her. just to clarify, my "dream girl" is not a model via the way she looks, because looks alone does not define my "dream girl". also, i am not

In [ ]:
# why did this match on lifestyle?
print(df.loc[407])

age                                                                   59
status                                                            single
sex                                                                    m
orientation                                                     straight
diet                                                            anything
drinks                                                          socially
drugs                                                              never
education                                   college/university_graduated
ethnicity                                                          other
height                                                              71.0
job                                         science / tech / engineering
location                                             hayward, california
offspring                                         no kids, no preference
pets                                               

In [ ]:
print(df.loc[392])

age                                                                   47
status                                                            single
sex                                                                    m
orientation                                                     straight
diet                                                             unknown
drinks                                                          socially
drugs                                                              never
education                                          high school_graduated
ethnicity                                                        unknown
height                                                              69.0
job                                                       rather not say
location                                       san francisco, california
offspring                                                        unknown
pets                                               

## Blending Scores

In [ ]:
df_gowers.iloc[[407]]

,age,status,sex,orientation,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,smokes,speaks
407,59,single,m,straight,anything,socially,never,college/university_graduated,other,71.0,science / tech / engineering,"hayward, california","no kids, no preference",dislikes pets,christianity,no,monolingual


In [ ]:
df.iloc[[407]]

,age,status,sex,orientation,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,smokes,speaks,speaks_original,combined_essay_cols,bert_embeddings
407,59,single,m,straight,anything,socially,never,college/university_graduated,other,71.0,science / tech / engineering,"hayward, california","no kids, no preference",dislikes pets,christianity,no,monolingual,english,hopefulromantic was taken and 2hopefulromantic...,"[-0.2053932398557663, 0.012036547996103764, -0..."


In [ ]:
# time to blend the functions!
def get_matches(user_index, df, df_gowers, essay_weight, lifestyle_weight):

  blended_score = []

  # get essay matching score
  essay_matches = get_essay_matches(user_index, df)  # gives us the (user_id, essay score)

  # get lifestyle matching score
  lifestyle_matches = get_lifestyle_matches(user_index, df_gowers)   # gives us the (user_id, lifestyle score)

  #essay table
  essay_dict = dict(essay_matches)
  lifestyle_dict = dict(lifestyle_matches)

  # getting keys
  keys = essay_dict.keys() | lifestyle_dict.keys()

  essay_score = [essay_dict[user_index] if user_index in essay_dict else 0.0 for user_index in keys]

  lifestyle_score = [lifestyle_dict[user_index] if user_index in lifestyle_dict else 0.0 for user_index in keys]


  # calculating final score
    final_score = (essay_score * essay_weight) + (lifestyle_score * lifestyle_weight)
    blended_score.append((user_index, final_score))

  top_matches = sorted(blended_score, key=lambda x: x[1], reverse=True)
  return top_matches

IndentationError: unexpected indent (<ipython-input-1-38be66444e94>, line 31)

In [ ]:
filtered_profiles = [profile for profile in user_profiles if (preferred_min_age <= profile["age"] and preferred_max_age >= profile["age"]) and (profile["location"] in preferred_locations) and (profile["orientation"] in preferred_orientations)]

In [ ]:
matches = [profile["user_index"] for profile in filtered_profiles, ]